In [6]:
# Загружаем необходимые библиотеки
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [15]:
# Парсинг новостей с сайта Хабр с использованием списка ключевых слов в заголовках статей (KEYWORDS)
KEYWORDS = ['Python', 'SQL', 'React', 'PHP', 'UX', 'Google']
URL = 'https://habr.com/ru/all/'
HEADERS = {"Accept": "ext/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
           "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:97.0) Gecko/20100101 Firefox/97.0"} 

In [16]:
# Выполняем GET запрос к сайту по указанному адресу URL
def get_html(url):
    r = requests.get(url, params = None, headers = HEADERS)
    return r

In [17]:
# Ищем заголовки новостных статей. Для заголовков, в которых встречаются ключевые слова из заданного списка,
# скачиваем текст статьи и формируем итоговый список по заданной структуре
def get_content_title(html):
    soup = BS(html.text, 'html.parser')
    result = []
    articles = soup.find_all("div", class_ = "tm-article-snippet")
    for article in articles:
        tag_time = article.find('time')
        article_time = tag_time.get('title')
        tag_h2 = article.find('h2')
        article_title = tag_h2.text
        tag_a = article.find('a', class_ = "tm-article-snippet__title-link")
        article_link = tag_a.get('href')
        for keyword in KEYWORDS:
            if keyword in article_title:
                article_url = 'https://habr.com' + article_link
                article_html = get_html(article_url)
                soup_a = BS(article_html.text, 'html.parser')
                tag_div = soup_a.find("div", class_ = "tm-article-body")
                article_content = tag_div.text.replace("\xa0", "").replace("\n", "")
                result.append((article_time, article_title, article_link, article_content))
                break
    return result

In [21]:
# Настраиваем формат вывода датафрейма, чтобы текст статьи отражался полностью
pd.set_option('display.max_colwidth', None)

In [32]:
# Парсинг сайта новостей и вывод результата в итоговый датасет
def parse():
    html = get_html(URL)
    if html.status_code == 200:
        news_list = get_content_title(html)
        if len(news_list) != 0:
            return news_list
        else:
            print('Ничего не найдено!!!')
    else:
        print(f'Error: {html.status_code}')
        
if __name__ == "__main__":
    news_list = parse()

df_news = pd.DataFrame(data = news_list, columns = ['Дата', 'Заголовок', 'Ссылка', 'Текст_статьи'])
df_news.head(10).style.set_properties(subset = 'Текст_статьи', **{'text-align': 'left'})    